In [2]:
import pandas as pd
import numpy as np
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/FintechDS

Mounted at /content/drive
/content/drive/My Drive/FintechDS


In [27]:
df = pd.read_csv("data/transformed/df.csv",index_col=False)
reviews_scraped = pd.read_csv("data/transformed/reviews_scraped.csv",index_col=False)
products_scraped = pd.read_csv("data/transformed/products_scraped.csv",index_col=False)

In [45]:
reviews_scraped.dropna(inplace= True)
products_scraped.dropna(inplace= True)

In [55]:
df.review_text = df.review_text.astype(str)
df.review_title = df.review_title.astype(str)
reviews_scraped.review_text = reviews_scraped.review_text.astype(str)

In [19]:
df.head()

,Unnamed: 0,rating,verified,PRODUCT_CATEGORY,product_id,review_title,review_text,cat_0,cat_1,cat_2,...,text_sentiment,text_subjectivity,rating_count,rating_avg,rating1,rating2,rating3,rating4,rating5,product_title
0,0,4.0,0.0,PC,B00008NG7N,useful,"When least you think so, this product will sav...",0.0,0.0,0.0,...,-0.300000,0.400000,233.0,4.6,0.03,0.01,0.07,0.07,0.81,"Targus PAUK10U Ultra Mini USB Keypad, Black"
1,1,4.0,1.0,Wireless,B00LH0Y3NM,New era for batteries,Lithium batteries are something new introduced...,0.0,0.0,0.0,...,0.219226,0.385320,214.0,3.8,0.17,0.06,0.14,0.11,0.53,Note 3 Battery : Stalion Strength Replacement ...
2,2,4.0,1.0,Wireless,B00LH0Y3NM,OK,"Seems to perform just ok, it's a battery. Came...",0.0,0.0,0.0,...,0.477778,0.611111,214.0,3.8,0.17,0.06,0.14,0.11,0.53,Note 3 Battery : Stalion Strength Replacement ...
3,3,3.0,0.0,Baby,B000I5UZ1Q,doesn't swing very well.,I purchased this swing for my baby. She is 6 m...,0.0,0.0,1.0,...,0.347143,0.600000,923.0,4.4,0.06,0.03,0.08,0.07,0.76,"Fisher-Price Papasan Cradle Swing, Starlight"
4,4,4.0,0.0,Office Products,B003822IRA,Great computing!,I was looking for an inexpensive desk calcolat...,0.0,0.0,0.0,...,-0.088542,0.441667,11026.0,4.8,0.01,0.01,0.03,0.10,0.85,Casio MS-80B Standard Function Desktop Calculator


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data split

In [69]:
from sklearn.model_selection import train_test_split

x = df.loc[:, df.columns != 'label']
y = df.label

x_train, x_val, y_train, y_val = train_test_split( x, y, test_size=0.2,shuffle=True, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5,shuffle=True, random_state=1)

# Model definition

In [13]:
class OurModelParameters():
    def __init__(self, bert):
      self.bert = bert
      pass

class OurModel(torch.nn.Module):

    def __init__(self, parameters: OurModelParameters):
      self.dropout = nn.Dropout(0.1)
      self.relu =  nn.ReLU()
      self.fc1 = nn.Linear(768,512)
      self.fc2 = nn.Linear(512,2)
      self.softmax = nn.LogSoftmax(dim=1)


      pass

    def forward(self, row_in_fake_real_products_df):
        """"
            row_in_fake_real_products_df is a df with the following columns:
                - product_title
                - review_title
                - rating1, rating2, rating3, rating4, rating5 (the percentages of ratings from each brackets, as values between 0.0 and 1.0)
                - 
        """
        pass
        

In [7]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 391 kB/s 
     |████████████████████████████████| 5.8 MB 35.6 MB/s 
     |████████████████████████████████| 1.3 MB 63.3 MB/s 
     |████████████████████████████████| 182 kB 71.8 MB/s 
     |████████████████████████████████| 7.6 MB 64.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=806a1f0ba3a774f27e1b12c17f493638b4392e0eab2eb48cca0386fd427370b9
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


# Text Embeddings

In [10]:
import transformers
from transformers import BertTokenizerFast,AutoTokenizer, BertTokenizer, AutoModel
#importing automodel didn't work with VSCode

bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [65]:
from tqdm.notebook import tqdm_notebook
def prepareDataForBert(data):
      tokenized = []

      pbar = tqdm_notebook(total = len(data))
      pbar.set_description(f'Tokenized: ')
      for i in data:
        tokenized.append(tokenizer.encode(i, add_special_tokens = True, truncation=True))
        pbar.update(1)
      pbar.close()
    
      max_len = 0
      for i in tokenized:
        if len(i) > max_len:
          max_len = len(i)

      padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized])
      attention_mask = np.where(padded != 0, 1, 0)
      
      return padded, attention_mask

# Scrapped review embeddings

In [67]:
padded_rscrapped, att_mask_rscrapped = prepareDataForBert(reviews_scraped.review_text)

  0%|          | 0/99098 [00:00<?, ?it/s]

In [75]:
padded_rscrapped_title, att_mask_rscrapped_title = prepareDataForBert(reviews_scraped.review_title)

  0%|          | 0/99098 [00:00<?, ?it/s]

# Normal Review embeddings (not scrapped)

In [70]:
padded_x_train_text, att_mask_x_train_text = prepareDataForBert(x_train.review_text)

  0%|          | 0/14485 [00:00<?, ?it/s]

In [71]:
padded_x_val_text, att_mask_x_val_text = prepareDataForBert(x_val.review_text)

  0%|          | 0/1811 [00:00<?, ?it/s]

In [72]:
padded_x_test_text, att_mask_x_test_text = prepareDataForBert(x_test.review_text)

  0%|          | 0/1811 [00:00<?, ?it/s]

# Product Title embeddings

In [74]:
padded_x_train_title, att_mask_x_train_title = prepareDataForBert(x_train.product_title)
padded_x_val_title, att_mask_x_val_title = prepareDataForBert(x_val.product_title)
padded_x_test_title, att_mask_x_test_title = prepareDataForBert(x_test.product_title)


  0%|          | 0/14485 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]